# Day 12: Garden Groups

https://adventofcode.com/2024/day/12

## --- Part One ---

In [99]:
import numpy as np
from pprint import pprint


file = 'input.txt'
# file = 'sample.txt'
# file = 'sample2.txt'
# file = 'sample3.txt'

# init vars
total_score = 0
garden = []
regions = []


# open file & load content
with open(file, 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        garden.append([x for x in line])


# re-use modified search from day 10
def dfs(matrix, i, j, visited, target_val,region_idx):
    global checked
    # Check if the current position is out of bounds or already visited
    if i < 0 or i >= len(matrix) or j < 0 or j >= len(matrix[0]) or visited[i][j] or matrix[i][j] == '.':
        return
    
    # if the value is not the target value exit
    if matrix[i][j] != target_val:
        return
            
    # Mark the current cell as visited
    visited[i][j] = True

    # if the target value is found append it to the correct region
    if matrix[i][j] == target_val:
        regions[region_idx].append([i,j])
        checked[i][j] = 1
    
    # Explore the neighbors (up, down, left, right)
    dfs(matrix, i - 1, j, visited, target_val,region_idx)  # Up
    dfs(matrix, i + 1, j, visited, target_val,region_idx)  # Down
    dfs(matrix, i, j - 1, visited, target_val,region_idx)  # Left
    dfs(matrix, i, j + 1, visited, target_val,region_idx)  # Right

# calculate the perimeter score
def calc_perimeter(plot):
    aaa = np.zeros_like(garden,dtype=int)
    perimiters = 0
    for loc in plot:
        aaa[loc[0]][loc[1]] = 1
        fences = 0
        # if no item ABOVE add 1 fence
        if [loc[0]-1,loc[1]] not in plot:
            fences +=1
        # if no item BELOW add 1 fence
        if [loc[0]+1,loc[1]] not in plot:
            fences +=1
        # if no item LEFT add 1 fence
        if [loc[0],loc[1]-1] not in plot:
            fences +=1
        # if no item RIGHT add 1 fence
        if [loc[0],loc[1]+1] not in plot:
            fences +=1
        perimiters += fences
    
    return perimiters

# numpy array of list(garden)
garden = np.array(garden)
# initialize a zeroes array of size (garden)
checked = np.zeros_like(garden,dtype=int)
# keep track of region nr
x = 0 
for row_nr,row in enumerate(garden):
    for col_nr,col in enumerate(row):
        # element is not checked already its a new region
        if checked[row_nr][col_nr] == 0:
            # init new visited array
            visited = np.zeros_like(garden,dtype=int)
            # append a new region
            regions.append([])
            # go through the garden and set the region
            dfs(garden, row_nr, col_nr, visited,garden[row_nr][col_nr],x)
            # up the region nr by 1
            x+=1

# for every plot in the region
for plot in regions:
    # calculate the perimeter
    plot_perimeter = calc_perimeter(plot)
    # calculate the plot score
    plot_score = len(plot)*plot_perimeter
    # up the total score
    total_score += plot_score

print('\nAnswer to part 1:', total_score)


Answer to part 1: 1473620


## --- Part Two ---

In [1]:
import numpy as np
from pprint import pprint


file = 'input.txt'
# file = 'sample.txt'
# file = 'sample2.txt'
# file = 'sample3.txt'

# init vars
total_score = 0
garden = []
regions = []


# open file & load content
with open(file, 'r') as f:
    lines = f.read().splitlines()
    for line in lines:
        garden.append([x for x in line])


# re-use modified search from day 10
def dfs(matrix, i, j, visited, target_val,region_idx):
    global checked
    # Check if the current position is out of bounds or already visited
    if i < 0 or i >= len(matrix) or j < 0 or j >= len(matrix[0]) or visited[i][j] or matrix[i][j] == '.':
        return
    
    # if the value is not the target value exit
    if matrix[i][j] != target_val:
        return
            
    # Mark the current cell as visited
    visited[i][j] = True

    # if the target value is found append it to the correct region
    if matrix[i][j] == target_val:
        regions[region_idx].append([i,j])
        checked[i][j] = 1
    
    # Explore the neighbors (up, down, left, right)
    dfs(matrix, i - 1, j, visited, target_val,region_idx)  # Up
    dfs(matrix, i + 1, j, visited, target_val,region_idx)  # Down
    dfs(matrix, i, j - 1, visited, target_val,region_idx)  # Left
    dfs(matrix, i, j + 1, visited, target_val,region_idx)  # Right

def calc_perimeter(plot):
    aaa = np.zeros_like(garden,dtype=int)
    perimiters = 0
    # keep a dict of the fence locations left/right/top/bottom
    fence_locations = {'left':[],'right':[],'top':[],'bottom':[]}
    # perimeters only down and 
    for loc in plot:
        aaa[loc[0]][loc[1]] = 1
        fences = 0
        if [loc[0]-1,loc[1]] not in plot: # up
            fences +=1
            fence_locations['top'].append(loc)
        if [loc[0]+1,loc[1]] not in plot: # down
            fences +=1
            fence_locations['bottom'].append(loc)
        if [loc[0],loc[1]-1] not in plot: # left
            fences +=1
            fence_locations['left'].append(loc)
        if [loc[0],loc[1]+1] not in plot: # right
            fences +=1            
            fence_locations['right'].append(loc)
        perimiters += fences
    
    # go through the fence location
    for key in fence_locations:
        scoop = fence_locations[key]
        # track the checked fences
        track = dict()
        # for top/bottom; check if there already is a fence on the same row directly to the left; if there is; subtract 1 from the total nr of fences
        if key == 'top' or key == 'bottom':
            for item in scoop:
                [row,col] = item
                if row in track:
                    track[row].append(col)
                else:
                    track[row] = [col]
            for row in track:
                for item in track[row]:
                    if (item-1) in track[row]:
                        perimiters -=1
        # for left/right; check if there already is a fence on the same col directly above; if there is; subtract 1 from the total nr of fences
        if key == 'left' or key == 'right':
            for item in scoop:
                [row,col] = item
                if col in track:
                    track[col].append(row)
                else:
                    track[col] = [row]
            for col in track:
                for item in track[col]:
                    if (item-1) in track[col]:
                        perimiters -=1

    # print('\n',aaa)
    return perimiters

# numpy array of list(garden)
garden = np.array(garden)
# initialize a zeroes array of size (garden)
checked = np.zeros_like(garden,dtype=int)
# keep track of region nr
x = 0 
for row_nr,row in enumerate(garden):
    for col_nr,col in enumerate(row):
        # element is not checked already its a new region
        if checked[row_nr][col_nr] == 0:
            # init new visited array
            visited = np.zeros_like(garden,dtype=int)
            # append a new region
            regions.append([])
            # go through the garden and set the region
            dfs(garden, row_nr, col_nr, visited,garden[row_nr][col_nr],x)
            # up the region nr by 1
            x+=1

# for every plot in the region
for plot in regions:
    # calculate the perimeter
    plot_perimeter = calc_perimeter(plot)
    # calculate the plot score
    plot_score = len(plot)*plot_perimeter
    # up the total score
    total_score += plot_score

print('Answer to part 2:', total_score)

Answer to part 2: 902620
